In [1]:
import pyscf
import pyscf.tools

In [2]:
molecule = """
 Cr 0.82627800 -1.30446200 -0.96524300
 Cr -0.82625500 1.30449100 0.96521400
 O 0.00001100 0.00001500 -0.00001400
 N 2.71379100 -0.58517300 -0.32156700
 H 3.52017600 -0.87998300 -0.88101600
 H 2.98302700 -0.84252100 0.62848200
 H 2.75216700 0.43162900 -0.34666000
 N 0.90611400 -0.01191500 -2.64373800
 H 0.15767300 0.67336300 -2.59732700
 H 0.79520000 -0.45289000 -3.56150100
 H 1.76766300 0.52845600 -2.74461100
 N 0.74644200 -2.59700900 0.71325300
 H 0.86887400 -3.59499400 0.51496700
 H -0.15371900 -2.54555600 1.18538900
 H 1.44183900 -2.41670700 1.43797600
 N -1.06123500 -2.02375100 -1.60891800
 H -1.51982700 -2.67507800 -0.96956400
 H -1.07760500 -2.52098500 -2.50431600
 H -1.71705200 -1.25706600 -1.73635500
 N 1.78821400 -2.82312800 -2.08895700
 H 2.20721200 -2.51946500 -2.97231100
 H 1.19687500 -3.61071400 -2.36865500
 H 2.56853800 -3.28036700 -1.60828700
 N -2.71376800 0.58520300 0.32153900
 H -2.72742600 -0.43151600 0.28926300
 H -3.01495100 0.88961900 -0.60504900
 H -3.51124000 0.82996900 0.91689300
 N -0.90609100 0.01194400 2.64371000
 H -0.86251400 0.45862400 3.56440800
 H -0.11862500 -0.63055500 2.63771800
 H -1.73984000 -0.57546100 2.70589400
 N 1.06125800 2.02378000 1.60888900
 H 1.08580100 2.47872200 2.52630400
 H 1.50134100 2.71049000 0.99382500
 H 1.72819800 1.26071800 1.68834600
 N -0.74641900 2.59703800 -0.71328200
 H -0.92607700 3.58837700 -0.52509900
 H -1.40673500 2.37764300 -1.45939400
 H 0.17080800 2.59240800 -1.15542700
 N -1.78819100 2.82315700 2.08892800
 H -1.19783000 3.61531400 2.35756800
 H -2.19798000 2.52398300 2.97800300
 H -2.57540900 3.27269300 1.61233200
"""

In [3]:
basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   6, # number of unpaired electrons
        charge  =   4,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"


symmetry:  C1


In [4]:

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()



******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 73  num. singly occ = 6
init E= -2722.30649096233
  HOMO = -0.180893429449332  LUMO = -0.0912350732357913
cycle= 1 E= -2721.1028545124  delta_E=  1.2  |g|= 1.19  |ddm|= 4.33
  HOMO = -0.484313365416376  LUMO = -0.390433229330059
cycle= 2 E= -2721.51059453132  delta_E= -0.408  |g|= 0.767  |ddm|= 1.34
  HOMO = -0.605933393834158  LUMO = -0.395171397843555
cycle= 3 E= -2721.64016454842  delta_E= -0.13  |g|= 0.325  |ddm|= 0.681
  HOMO = -0.651450051333488  LUMO = -0.390226849063734
cycle= 4 E= -2721.66317490064  delta_E= -0.023  |g|= 0.105  |ddm|= 0.295
  HOMO = -0.626041834704189  LUMO = -

In [5]:
F = mf.get_fock()

In [78]:
import numpy as np
import scipy
import copy as cp
import math


def sym_ortho(frags, S, thresh=1e-8):
    """
    frags is a list of mo-coeff matrices
    """
    Nbas = S.shape[1]
    
    inds = []
    Cnonorth = np.hstack(frags)
    shift = 0
    for f in frags:
        inds.append(list(range(shift, shift+f.shape[1])))
        shift += f.shape[1]
        
    
    Smo = Cnonorth.T @ S @ Cnonorth
    X = np.linalg.inv(scipy.linalg.sqrtm(Smo))
    # print(Cnonorth.shape, X.shape)
    Corth = Cnonorth @ X
    
    frags2 = []
    for f in inds:
        frags2.append(Corth[:,f])
    return frags2


In [79]:
import scipy

# Find AO's corresponding to atoms 
# print(mf.mol.aoslice_by_atom())
# print(mf.mol.ao_labels(fmt=False, base=0))
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d", "4s"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("2s", "2p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3d", "4s"):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)



[[3, 14, 15, 16, 17, 18], [63, 65, 66, 67], [34, 45, 46, 47, 48, 49]]


In [80]:
def svd_orbs(C, frag, S, thresh=1e-7, verbose=2):
    print(" Frag: ", frag)
    X = scipy.linalg.sqrtm(S)
    Cfrag = X@C
    
    _,s,V = np.linalg.svd(Cfrag[frag,:], full_matrices=True)
    print("Singular Values")
    for i in range(len(s)):
        print(" %4i : %12.8f" %(i, s[i]))
    return C @ V.T

def rmv_svd_orbs(C, frag, S, thresh=1e-7, verbose=2):
    print(" Frag: ", frag)
    X = scipy.linalg.sqrtm(S)
    Xinv = scipy.linalg.inv(X)

    proj_basis = X[:,frag]
    Cfrag = proj_basis.T @ C
    
    U,s,V = np.linalg.svd(Cfrag, full_matrices=True)
    print("Singular Values")
    for i in range(len(s)):
        print(" %4i : %12.8f" %(i, s[i]))
    return proj_basis @ U

C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing
# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]
print(ndocc, nsing, nvirt)
print(Cdocc.shape)
print(Csing.shape)
print(Cvirt.shape)

print(" Doubly Occupied")
Cdocc = svd_orbs(Cdocc, full, S)
print(" Doubly Unoccupied")
Cvirt = svd_orbs(Cvirt, full, S)

# Ctmp = rmv_svd_orbs(Csing, full, S)

pyscf.tools.molden.from_mo(mf.mol, "Cdocc.molden", Cdocc)
pyscf.tools.molden.from_mo(mf.mol, "Cvirt.molden", Cvirt)
# pyscf.tools.molden.from_mo(mf.mol, "Ctmp.molden", Ctmp)

73 6 287
(366, 73)
(366, 6)
(366, 287)
 Doubly Occupied
 Frag:  [3, 14, 15, 16, 17, 18, 34, 45, 46, 47, 48, 49, 63, 65, 66, 67]
Singular Values
    0 :   0.73870673
    1 :   0.69762751
    2 :   0.69743161
    3 :   0.69445123
    4 :   0.36995973
    5 :   0.36317037
    6 :   0.24960301
    7 :   0.24831048
    8 :   0.20187650
    9 :   0.17407653
   10 :   0.03274360
   11 :   0.03261201
   12 :   0.03038757
   13 :   0.03028218
   14 :   0.02765145
   15 :   0.02757821
 Doubly Unoccupied
 Frag:  [3, 14, 15, 16, 17, 18, 34, 45, 46, 47, 48, 49, 63, 65, 66, 67]
Singular Values
    0 :   0.98473213
    1 :   0.97941099
    2 :   0.96868051
    3 :   0.96834827
    4 :   0.93172278
    5 :   0.92904780
    6 :   0.71953982
    7 :   0.71728448
    8 :   0.71708714
    9 :   0.67402704
   10 :   0.34764231
   11 :   0.34668957
   12 :   0.34375203
   13 :   0.34350825
   14 :   0.30347675
   15 :   0.30332149


# Possible Active Spaces

These orbitals could be divided up into a series of well-defined active spaces: `(O,S,V)`
1. Active Space 16: `(4,6,6)`
    - **Cr:** 3d, 4s
    - **O:**  2p, 2s 
2. Active Space 20: `(4,6,10)`
    - **Cr:** 3d, 4s
    - **O:**  2p, 2s, 3p, 3s  
3. Active Space 26: `(4,6,16)`
    - **Cr:** 3d, 4s, 4d, 5s
    - **O:**  2p, 2s, 3p, 3s  
4. Active Space 32: `(10,6,16)`
    - **Cr:** 3d, 4s, 4d, 5s, Cr-N sigma
    - **O:**  2p, 2s, 3p, 3s  
5. Active Space 38: `(16,6,16)`
    
    Starts to extend onto NH3 groups due to overlap with he target 4s
    - **Cr:** 3d, 4s, 4d, 5s, Cr-N sigma
    - **O:**  2p, 2s, 3p, 3s  
    - **N:**  N-H bonds 

# Active Space 16

We will start with just the orbitals most directly related to the user defined `valence space`

In [84]:
nact_docc = 4
nact_virt = 6

nfrzn = Cdocc.shape[1] - nact_docc
Cenv = Cdocc[:,nact_docc:-1]
Cact = np.hstack((Cdocc[:,0:nact_docc], Csing, Cvirt[:,0:nact_virt]))
print(Cenv.shape)
print(Cact.shape)
print("Should be zero: ", np.trace(Cenv.T@S@Cact@Cact.T@Cenv))

Cfrag1 = svd_orbs(Cact, frag1, S)[:,0:6]
Cfrag2 = svd_orbs(Cact, frag2, S)[:,0:4]
Cfrag3 = svd_orbs(Cact, frag3, S)[:,0:6]

Cfrag1, Cfrag2, Cfrag3 = sym_ortho((Cfrag1, Cfrag2, Cfrag3), S)

pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)

pyscf.tools.molden.from_mo(mf.mol, "Cfrag1.molden", Cfrag1)
pyscf.tools.molden.from_mo(mf.mol, "Cfrag2.molden", Cfrag2)
pyscf.tools.molden.from_mo(mf.mol, "Cfrag3.molden", Cfrag3)

Ctot = np.hstack((Cfrag1, Cfrag2, Cfrag3))
pyscf.tools.molden.from_mo(mf.mol, "Ctot.molden", Ctot)


(366, 68)
(366, 16)
Should be zero:  -1.4399014857672325e-16
 Frag:  [3, 14, 15, 16, 17, 18]
Singular Values
    0 :   0.96851041
    1 :   0.95615989
    2 :   0.94265701
    3 :   0.94257732
    4 :   0.93723667
    5 :   0.92064148
 Frag:  [63, 65, 66, 67]
Singular Values
    0 :   0.79998564
    1 :   0.72422779
    2 :   0.70494776
    3 :   0.70469377
 Frag:  [34, 45, 46, 47, 48, 49]
Singular Values
    0 :   0.96851816
    1 :   0.95615798
    2 :   0.94266132
    3 :   0.94257629
    4 :   0.93723640
    5 :   0.92063933


In [85]:
clusters = [list(range(0,6)), list(range(6,10)), list(range(10,16))]
init_fspace = [(3,0), (4,4), (3,0)]
print(clusters)
print(init_fspace)

[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9], [10, 11, 12, 13, 14, 15]]
[(3, 0), (4, 4), (3, 0)]


# Make Integrals

In [86]:
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

In [87]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Ctot.T @ h @ Ctot;
j = Ctot.T @ j @ Ctot;
k = Ctot.T @ k @ Ctot;

In [88]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Ctot, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [89]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Ctot @ Ctot.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [90]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Ctot)
np.save("overlap_mat", S)

In [73]:
import numpy as np
Ccmf = np.load("Ccmf.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf.molden", Ccmf)


In [91]:
print(Cdocc.shape)
print(Csing.shape)
P = 2*Cdocc@Cdocc.T + Csing@Csing.T
print(np.trace(P@S))
print(Ctot.shape)


(366, 73)
(366, 6)
152.00000000000006
(366, 16)


In [75]:
Ptot = Ctot @ Ctot.T
Pact = Cact @ Cact.T 
Penv = Cenv @ Cenv.T
Psing = Csing @ Csing.T
Pdocc = Cdocc @ Cdocc.T
Pvirt = Cvirt @ Cvirt.T

3.8094056520453717e-28